In [1]:
import pandas as pd
import sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pyodbc

In [2]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'ODBC Driver 17 for SQL Server',
 'ODBC Driver 13 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)']

# Using SQLALchemy for Cnxn

In [3]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=NJDPatient1.Citymd.local;"
            "Database=Summit;"
            "Trusted_Connection=yes;")
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": cnxn_str})

from sqlalchemy import create_engine
engine = create_engine(connection_url)


In [4]:
ex = engine.connect()
sql =" \
select a.patientid,a.firstname,a.lastname,a.dob,a.sex,a.insurancepackageid,eligibilityservicedate, \
                      eligibilitystatus,SequenceNumber,\
                      row_number() over (partition by patientid order by eligibilityservicedate desc) as rn \
                      into ##temp1 \
                      from [Summit].[dbo].[Athena_PatientInsurance] a \
                      where  eligibilitystatus = 'Y' AND SequenceNumber = 1"
                                
ex.execute(sql)

In [ ]:
ex.close
ex = engine.connect()
temp = pd.read_sql_query("select * from ##temp1 where rn = 1", engine)
temp

In [ ]:
ex = engine.connect()
sql = "Select * into ##temp2 from ##temp1 where rn = 1"
ex.execute(sql)
ex.close

In [ ]:
ex = engine.connect()

sql = pd.read_sql_query(" \
select distinct a.patientid,a.firstname,a.lastname,a.dob,a.sex,b.custominsurancegrouping,a.eligibilityservicedate,\ 
map.payerCategory,\
eligibilitystatus,\
SequenceNumber,\
c.MRN, \
Planpayer,\
Startdate,d.serviceline,d.engagement, \
d.lastvisitdate,LastPCvisitDate,PCP_Reason_Detail,Attributed_YN\
from ##temp2 as  a\
join [Summit].[dbo].[Athena_Payer] b on a.insurancepackageid = b. insurancepackageid \
left join VBC.dbo.PayerMapping map with(nolock) on map.InsurancePackageID = b.InsurancePackageID \
left join VBC.dbo.VBCAttributions c on a.patientid = c.enterpriseid \
left join VBC.dbo.PatSummitProviderEngagement d on a.patientid = d.enterpriseid \
where b.custominsurancegrouping = 'AETNA MEDICARE' \
AND eligibilitystatus = 'Y' AND SequenceNumber = 1 \
",engine)



In [ ]:
sql

In [ ]:
#cursor.execute(sql)

In [ ]:
#data = cursor.fetchall()

In [ ]:
data

In [ ]:
ex = engine.connect()
sql = "DROP TABLE ##temp1"
ex.execute(sql)